# Prompt + LLM 

This is a practice for using langchain Prompt. 

## PromptTemplate + LLM 

In [3]:
import os
from dotenv import load_dotenv

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("Provide a short joke about {topic}")
model = ChatOpenAI()
chain = prompt | model

In [5]:
chain.invoke({"topic": "programming"})

AIMessage(content='Why do programmers prefer dark mode? \nBecause light attracts bugs!')

In [6]:
chain = prompt | model.bind(stop=["\n"])

In [7]:
chain.invoke({"topic": "programming"})

AIMessage(content='Why do programmers prefer dark mode? ')

#### Attaching Function call information

In [8]:
functions = [
    {
        "name": "joke",
        "description": "A joke",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "The setup for the joke"},
                "punchline": {
                    "type": "string",
                    "description": "The punchline for the joke",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]
chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)

In [9]:
chain.invoke({"topic": "smartphone"}, config={})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "setup": "Why did the smartphone bring a ladder?",\n  "punchline": "Because it wanted to reach the cloud!"\n}', 'name': 'joke'}})

## PromptTemplate + LLM + OutputParser

In [10]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()

In [11]:
chain.invoke({"topic": "programming"})

'Why do programmers prefer dark mode?\n\nBecause light attracts bugs!'

### Function Output Parser

In [12]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonOutputFunctionsParser()
)

In [13]:
chain.invoke({"topic": "programming"})

{'setup': 'Why do programmers prefer dark mode?',
 'punchline': 'Because light attracts bugs!'}

In [14]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [15]:
chain.invoke({"topic": "programming"})

'Why do programmers prefer dark mode?'

## Simplifying input

Using `RunnablePassthrough` function, pass `topic` as arguement in `RunnableParallel`

In [16]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

map_ = RunnableParallel(topic=RunnablePassthrough())
chain = (
    map_
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [17]:
chain.invoke("programming")

'Why do programmers prefer dark mode?'

In [18]:
chain = (
    {"topic": RunnablePassthrough()}
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [19]:
chain.invoke("programming")

'Why do programmers prefer dark mode?'